In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Import Dependancies

In [3]:
#!/usr/bin/env python3.11
!python --version

import mesa
print(f"mesa version: {mesa.__version__}")
import numpy as np
import pandas as pd
import seaborn as sns

Python 3.12.6
mesa version: 3.1.5


# The Naming Game

text

In [4]:
# the naming game implementation
# test change

#mesa scheduler

#mesa agent

#mesa data analysis

Create Random Word

In [5]:
from random import choice
from string import ascii_uppercase

def create_word(min_length=6, max_length=10):
    new_word = ''.join(choice(ascii_uppercase) for _ in range(np.random.randint(min_length, max_length)))
    return new_word

Create Agent

In [12]:
from random import choice

class NamingGameAgent(mesa.Agent):
    def __init__(self, model, m_objects=1):
        super().__init__(model)
        self.m_objects_ = m_objects
        self.memory = dict()

    def choose_object_word(self):
        chosen_object = choice(range(self.m_objects_))
        if chosen_object in self.memory.keys():
            chosen_word = choice(self.memory[chosen_object])
        else:
            chosen_word = create_word()
            self.memory[chosen_object] = [chosen_word]
        return chosen_object, chosen_word

    def recognize_word(self, chosen_object, chosen_word):
        if chosen_object not in self.memory.keys():
            return False
        if chosen_word in self.memory[chosen_object]:
            return True
        else:
            return False

    def update_memory(self, chosen_object, chosen_word):
        self.memory[chosen_object] = [chosen_word]

    def add_word(self, chosen_object, chosen_word):
        if chosen_object not in self.memory.keys():
            self.memory[chosen_object] = [chosen_word]
        self.memory[chosen_object].append(chosen_word)

Create Model

In [13]:
from mesa.datacollection import DataCollector
from random import sample

class NamingGameModel(mesa.Model):
    def __init__(self, N=100, m_objects=1, seed=None):
        super().__init__(seed=seed)
        self.num_agents = N
        #TODO figure out schedulte stuff self.schedule = mesa.time.RandomActivation(self) if needed
        #self.running = True
        #TODO make datacollection better
        self.datacollector = DataCollector(
            agent_reporters={"Memory": "memory"}
        )
        self.m_objects = m_objects
        NamingGameAgent.create_agents(model=self, n=N, m_objects=m_objects)
    
    def step(self):
        self.datacollector.collect(self)
        speaker, hearer = sample(self.agents, 2)
        chosen_object, chosen_word = speaker.choose_object_word()
        is_successful = hearer.recognize_word(chosen_object, chosen_word)
        if is_successful:
            speaker.update_memory(chosen_object, chosen_word)
            hearer.update_memory(chosen_object, chosen_word)
        else:
            hearer.add_word(chosen_object, chosen_word)


Run Model

In [14]:
N = 100
M = 1
t = 10
model = NamingGameModel(N, M)
# run the model for 10 steps
for i in range(t):
    model.step()
    #TODO datacollection here and after all runs